# Deep Hedging in Incomplete Markets — GBM & Heston

**MSc Thesis Experiment Runner**

Runs the full deep hedging pipeline under two market dynamics:
- **GBM** (constant volatility, calibrated to S&P 500)
- **Heston** (stochastic volatility, calibrated to S&P 500)

## Setup
1. **Runtime → Change runtime type → A100 GPU** (Pro+ recommended)
2. Click **Connect**
3. Run **Cell 1** (clone + install)

## Two ways to run
- **Option A (Browser):** Run cells directly in this notebook
- **Option B (VS Code):** Run Cell 2 to get an SSH tunnel, then connect VS Code and use the terminal

## Checkpoint / Resume
All progress is automatically checkpointed (Optuna trials in SQLite, per-seed metrics, cached features). If the runtime disconnects mid-run:
1. Reconnect and re-run **Cell 1** (re-clone + install)
2. Re-run the **same experiment cell** — it skips completed work and picks up where it left off

## Safe practice for long runs
For multi-hour or overnight training:
- **Print/log progress regularly.** The pipeline prints each Optuna trial, seed, and stage as it completes — watch the output to monitor progress.
- **Save checkpoints to Google Drive.** Copy the `outputs/` directory to Drive periodically so results survive runtime recycling (see Cell 2b below).
- **Assume sessions can still end and plan to resume.** Even with Pro+, Colab may reclaim GPUs after ~12 hours. The checkpoint system ensures no work is lost — just reconnect and re-run.

In [12]:
# Cell 1: Clone repo and install dependencies
!git clone https://github.com/thabangTheActuaryCoder/deep-hedging-thesis.git
%cd deep-hedging-thesis
!pip install -q torch numpy matplotlib optuna sqlalchemy

import torch
print(f'\nPython: {__import__("sys").version}')
print(f'PyTorch: {torch.__version__}')
print(f'GPU available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'Device: {torch.cuda.get_device_name(0)}')
    mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'Memory: {mem:.1f} GB')

Cloning into 'deep-hedging-thesis'...
remote: Enumerating objects: 217, done.
remote: Counting objects: 100% (217/217), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 217 (delta 95), reused 198 (delta 78), pack-reused 0 (from 0)
Receiving objects: 100% (217/217), 4.08 MiB | 17.42 MiB/s, done.
Resolving deltas: 100% (95/95), done.
/content/deep-hedging-thesis/deep-hedging-thesis

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch: 2.9.0+cu126
GPU available: True
Device: NVIDIA A100-SXM4-40GB
Memory: 42.5 GB


In [13]:
# Cell 2b (optional): Mount Google Drive and sync checkpoints
# Run this BEFORE the experiment to back up outputs to Drive automatically.
# On resume after disconnect, it restores outputs from Drive so nothing is lost.

from google.colab import drive
import shutil, os

drive.mount('/content/drive')

DRIVE_BACKUP = '/content/drive/MyDrive/deep_hedging_outputs'
LOCAL_OUTPUTS = '/content/deep-hedging-thesis/outputs'

# Restore from Drive if outputs exist there but not locally
if os.path.exists(DRIVE_BACKUP) and not os.path.exists(LOCAL_OUTPUTS):
    print('Restoring outputs from Google Drive...')
    shutil.copytree(DRIVE_BACKUP, LOCAL_OUTPUTS)
    print(f'Restored {sum(len(f) for _, _, f in os.walk(LOCAL_OUTPUTS))} files')

def backup_to_drive():
    """Call this periodically or after the experiment to save outputs to Drive."""
    if os.path.exists(LOCAL_OUTPUTS):
        if os.path.exists(DRIVE_BACKUP):
            shutil.rmtree(DRIVE_BACKUP)
        shutil.copytree(LOCAL_OUTPUTS, DRIVE_BACKUP)
        print(f'Backed up outputs to {DRIVE_BACKUP}')

print('Drive mounted. Call backup_to_drive() anytime to save outputs.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted. Call backup_to_drive() anytime to save outputs.


In [14]:
# Cell 3: Sanity check — all tests should pass
!python -m pytest tests/test_validation.py -v

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/deep-hedging-thesis/deep-hedging-thesis
plugins: anyio-4.12.1, typeguard-4.4.4, langsmith-0.6.8
collected 13 items                                                             

tests/test_validation.py::TestNoLookAhead::test_base_features_no_lookahead PASSED [  7%]
tests/test_validation.py::TestNoLookAhead::test_signature_features_no_lookahead PASSED [ 15%]
tests/test_validation.py::TestNoLookAhead::test_signature_at_zero PASSED [ 23%]
tests/test_validation.py::TestSelfFinancing::test_portfolio_update PASSED [ 30%]
tests/test_validation.py::TestSelfFinancing::test_path_terminal_consistency PASSED [ 38%]
tests/test_validation.py::TestSelfFinancing::test_no_future_prices_in_delta PASSED [ 46%]
tests/test_validation.py::TestReproducibility::test_market_simulation PASSED [ 53%]
tests/test_

In [ ]:
# Cell 5 (FULL RUN): both GBM + Heston, 100k paths, ~4-8 hours on A100
# Safe to re-run after disconnect — automatically resumes from checkpoints
!python run_experiment.py \
    --paths 100000 \
    --N 200 \
    --epochs 1000 \
    --patience 15 \
    --batch_size 2048 \
    --n_trials 60 \
    --seeds 0 1 2 3 4 \
    --substeps 0 5 10 \
    --market_model both

# Auto-backup to Drive when experiment finishes (requires Cell 2b)
try:
    backup_to_drive()
except NameError:
    print('Tip: run Cell 2b first to enable automatic Google Drive backups')

Device: cuda
Quick mode: False
Market model: both
Paths=100000  Split hash=0d2ec472ec087faf
Train=60000  Val=20000  Test=20000

  MARKET MODEL: GBM (calibrated)
  GBM: r=0.043, vols=[0.18, 0.22], extra_vol=0.06

  Pipeline: GBM market  (m_brownian=3)

=== [gbm] Step 2: Feature Construction ===
  Feature dim: 24

=== [gbm] Step 3: Stage 1 – Optuna HP Search (TPE) ===

--- FNN ---

  Optuna search for FNN: up to 60 trials (search space = 108 configs)
    Trial 0: depth=5 width=256 act=alt_relu_tanh lr=0.0003 -> CVaR95=0.074074  MSE=0.002298
    Trial 1: depth=3 width=128 act=tanh_all lr=0.0003 -> CVaR95=0.069650  MSE=0.002441
    Trial 2: depth=5 width=256 act=alt_tanh_relu lr=0.003 -> CVaR95=0.073402  MSE=0.002483
    Trial 3: depth=3 width=64 act=relu_all lr=0.001 -> CVaR95=0.071358  MSE=0.002318
    Trial 4: depth=3 width=128 act=relu_all lr=0.0003 -> CVaR95=0.072266  MSE=0.002270
    Trial 5: depth=5 width=256 act=tanh_all lr=0.001 -> CVaR95=0.068911  MSE=0.002549
    Trial 6: depth=

In [ ]:
# Cell 6: Preview GBM validation plots
from IPython.display import Image, display
import glob

print('=== GBM Validation Plots ===')
for img in sorted(glob.glob('outputs/gbm/plots_val/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 7: Preview Heston validation plots
from IPython.display import Image, display
import glob

print('=== Heston Validation Plots ===')
for img in sorted(glob.glob('outputs/heston/plots_val/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 8: Heston stochastic volatility diagnostic plots
from IPython.display import Image, display
import glob

print('=== Heston Stochastic Volatility Diagnostics ===')
for img in sorted(glob.glob('outputs/heston/plots_heston/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 9: GBM vs Heston comparison plots
from IPython.display import Image, display
import glob

print('=== GBM vs Heston Comparison ===')
for img in sorted(glob.glob('outputs/comparison/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 10: 3D delta surface plots
from IPython.display import Image, display
import glob

for label, pattern in [('GBM', 'outputs/gbm/plots_3d/*.png'),
                       ('Heston', 'outputs/heston/plots_3d/*.png')]:
    imgs = sorted(glob.glob(pattern))
    if imgs:
        print(f'\n=== {label} 3D Delta Surfaces ===')
        for img in imgs:
            print(f'\n--- {img} ---')
            display(Image(filename=img, width=700))

In [ ]:
# Cell 11: Show validation metrics (both market models)
import json, os

for market in ['gbm', 'heston']:
    path = f'outputs/{market}/val_metrics.json'
    if not os.path.exists(path):
        continue
    with open(path) as f:
        metrics = json.load(f)
    print(f'\n{"="*50}')
    print(f'  {market.upper()} — Best model: {metrics["best_model"]}')
    print(f'{"="*50}')
    for model, agg in metrics['aggregated_val_metrics'].items():
        cvar = agg['CVaR95_shortfall']
        mse = agg['MSE']
        print(f'  {model:6s}  CVaR95 = {cvar["mean"]:.6f} +/- {cvar["std"]:.6f}  '
              f'MSE = {mse["mean"]:.6f} +/- {mse["std"]:.6f}')

summary_path = 'outputs/metrics_summary.json'
if os.path.exists(summary_path):
    with open(summary_path) as f:
        combined = json.load(f)
    print(f'\n{"="*50}')
    print('  COMBINED SUMMARY')
    print(f'{"="*50}')
    for market, agg in combined.items():
        print(f'\n  [{market.upper()}]')
        for model, m in agg.items():
            cvar = m.get('CVaR95_shortfall', {})
            if isinstance(cvar, dict):
                print(f'    {model:6s}  CVaR95 = {cvar.get("mean",0):.6f} +/- {cvar.get("std",0):.6f}')

In [ ]:
## Replot (optional)

If you want to adjust figure dimensions, colors, grids, or fonts without
rerunning the experiment, edit the `STYLE` dict in `replot.py` and rerun
Cell 13 below. The data was saved during the experiment.

# Cell 13: Regenerate comparison plots from saved data (edit STYLE in replot.py first)
!python replot.py --data outputs/comparison/comparison_data.pt \
                  --metrics outputs/metrics_summary.json \
                  --out outputs/comparison

from IPython.display import Image, display
import glob
for img in sorted(glob.glob('outputs/comparison/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 14: Download all outputs as zip
import shutil
from google.colab import files

shutil.make_archive('outputs', 'zip', '.', 'outputs')
files.download('outputs.zip')